In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
from tqdm import tqdm

def load_and_preprocess_data(file_path, scaler_type="robust", test_size=0.2, random_state=42):
    """
    Load and preprocess the data with multiple scaler options.
    
    Args:
        file_path: Path to the CSV file
        scaler_type: Type of scaler to use ('standard', 'robust', or 'minmax')
        test_size: Proportion of data to use for testing
        random_state: Random seed for reproducibility
        
    Returns:
        train_tensor: Tensor of training data
        test_tensor: Tensor of test data
        scaler: Fitted scaler for future use
    """
    df = pd.read_csv(file_path, index_col=0)
    data = df.to_numpy()
    
    # Split data into train and test sets
    train_data, test_data = train_test_split(data, test_size=test_size, random_state=random_state)
    
    # Select scaler based on input
    if scaler_type == "standard":
        scaler = StandardScaler()
    elif scaler_type == "robust":
        scaler = RobustScaler()
    elif scaler_type == "minmax":
        scaler = MinMaxScaler()
    else:
        raise ValueError("Scaler type must be 'standard', 'robust', or 'minmax'")
    
    # Scale the data
    train_data = scaler.fit_transform(train_data)
    test_data = scaler.transform(test_data)
    
    # Feature engineering: add interaction terms for selected features
    # For demonstration, we'll create interactions between every 5th feature
    # In practice, you would use domain knowledge to select meaningful interactions
    
    # Convert to tensors
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_tensor = torch.tensor(train_data, dtype=torch.float32).to(device)
    test_tensor = torch.tensor(test_data, dtype=torch.float32).to(device)
    
    return train_tensor, test_tensor, scaler, device


In [27]:
class EnhancedTransformerModel(nn.Module):
    def __init__(self, num_features, d_model=128, num_heads=8, num_layers=4, 
                 dropout=0.1, dim_feedforward=512, activation="gelu"):
        super(EnhancedTransformerModel, self).__init__()
        
        # Initial embedding layer for each feature
        self.embedding = nn.Linear(1, d_model)
        
        # Column embedding for feature position information
        self.column_embedding = nn.Embedding(num_features, d_model)
        
        # Domain-specific initialization for embeddings (if available)
        # Here we're using a simple uniform initialization as placeholder
        nn.init.xavier_uniform_(self.embedding.weight)
        nn.init.xavier_uniform_(self.column_embedding.weight)
        
        # Layer normalization before transformer
        self.pre_norm = nn.LayerNorm(d_model)
        
        # Create stacked transformer layers with enhanced configuration
        encoder_layers = []
        for _ in range(num_layers):
            encoder_layer = nn.TransformerEncoderLayer(
                d_model=d_model,
                nhead=num_heads,
                dim_feedforward=dim_feedforward,
                dropout=dropout,
                activation=activation,
                batch_first=True
            )
            encoder_layers.append(encoder_layer)
        
        self.transformer_encoder_layers = nn.ModuleList(encoder_layers)
        
        # Additional residual layers
        self.residual_projections = nn.ModuleList([
            nn.Linear(d_model, d_model) for _ in range(num_layers-1)
        ])
        
        # Output projection
        self.output_layer = nn.Linear(d_model, 1)
        
        # Initialize output layer with small weights
        nn.init.xavier_uniform_(self.output_layer.weight, gain=0.1)
        nn.init.zeros_(self.output_layer.bias)
        
    def forward(self, x, column_indices, mask=None, missing_mask=None):
        # Unsqueeze to add feature dimension
        x = x.unsqueeze(-1)
        
        # Embed each feature
        x_embed = self.embedding(x)
        
        # Add column embeddings
        column_embed = self.column_embedding(column_indices)
        x_embed = x_embed + column_embed.unsqueeze(0)
        
        # Apply layer normalization
        x_embed = self.pre_norm(x_embed)
        
        # Pass through transformer layers with residual connections
        x_encoded = x_embed
        
        # Store intermediate outputs for residual connections
        intermediate_outputs = []
        
        for i, encoder_layer in enumerate(self.transformer_encoder_layers):
            # Apply transformer layer
            layer_output = encoder_layer(x_encoded, src_mask=mask)
            
            # For all but the first layer, add a residual connection from earlier layers
            if i > 0:
                # Apply residual projection
                for j, prev_output in enumerate(intermediate_outputs):
                    projected = self.residual_projections[j](prev_output)
                    layer_output = layer_output + projected
            
            x_encoded = layer_output
            intermediate_outputs.append(x_encoded)
            
        # Final output projection
        output = self.output_layer(x_encoded)
        
        # Handle missing values with more sophisticated approach
        if missing_mask is not None:
            # For missing values, we can apply a separate processing
            # Here we're just implementing a placeholder that could be enhanced
            # with domain-specific logic
            missing_indices = missing_mask == 1
            # You could add special handling for missing values here
        
        return output.squeeze(-1)

In [29]:
def create_missing_mask(data, missing_fraction=0.2, strategy="random"):
    """
    Create a missing value mask with different strategies.
    
    Args:
        data: Input tensor
        missing_fraction: Fraction of values to mask
        strategy: Strategy for masking ('random', 'structured', 'column')
        
    Returns:
        mask: Binary mask where 1 indicates missing value
    """
    if strategy == "random":
        # Completely random masking (MCAR)
        mask = torch.rand(data.shape).to(data.device) < missing_fraction
        
    elif strategy == "structured":
        # Structured masking - values more likely to be missing if they're high (MNAR)
        # This creates a pattern where higher values are more likely to be masked
        probs = torch.sigmoid((data - data.mean()) / data.std() * 2)
        mask = torch.rand(data.shape).to(data.device) < (probs * missing_fraction * 2)
        
        # Ensure we have approximately the right number of masked values
        current_frac = mask.float().mean().item()
        if current_frac > 0:
            mask = mask * (missing_fraction / current_frac)
            mask = torch.rand(data.shape).to(data.device) < mask
        
    elif strategy == "column":
        # Column-based masking - some columns missing more than others
        mask = torch.zeros_like(data, dtype=torch.bool)
        num_cols = data.shape[1]
        
        # Randomly assign higher missing probability to some columns
        col_probs = torch.ones(num_cols).to(data.device)
        high_miss_cols = torch.randperm(num_cols)[:num_cols//3]
        col_probs[high_miss_cols] = 3.0
        
        # Normalize to get the right overall fraction
        col_probs = col_probs * (missing_fraction * num_cols / col_probs.sum())
        
        # Apply column-specific missing probabilities
        for col in range(num_cols):
            mask[:, col] = torch.rand(data.shape[0]).to(data.device) < col_probs[col]
    
    else:
        raise ValueError("Strategy must be 'random', 'structured', or 'column'")
    
    return mask.int()

In [30]:
class EnhancedImputableLoss(nn.Module):
    def __init__(self, alpha=1.0, beta=0.1):
        super(EnhancedImputableLoss, self).__init__()
        self.alpha = alpha  # Weight for MSE loss
        self.beta = beta    # Weight for consistency loss
        
    def forward(self, predictions, ground_truth, mask):
        """
        Compute a multi-component loss for imputation.
        
        Args:
            predictions: Model predictions
            ground_truth: True values
            mask: Binary mask where 1 indicates missing/masked values
            
        Returns:
            loss: Combined loss value
        """
        # 1. MSE loss on masked values
        mse_loss = nn.MSELoss(reduction='none')
        element_loss = mse_loss(predictions, ground_truth)
        masked_mse = (element_loss * mask).sum() / (mask.sum() + 1e-8)
        
        # 2. Consistency loss - predictions should be consistent with observed values
        # For columns with both observed and missing values, the distributions should be similar
        col_means_pred = predictions.mean(dim=0)
        col_means_true = ground_truth.mean(dim=0)
        consistency_loss = F.mse_loss(col_means_pred, col_means_true)
        
        # Combine losses
        combined_loss = self.alpha * masked_mse + self.beta * consistency_loss
        
        return combined_loss

In [31]:
def evaluate_model(model, test_tensor, criterion, column_indices, missing_fraction=0.2, masking_strategy="random"):
    """
    Evaluate the model on test data.
    
    Args:
        model: Trained model
        test_tensor: Test data
        criterion: Loss function
        column_indices: Feature indices
        missing_fraction: Fraction of values to mask
        masking_strategy: Strategy for creating missing values
        
    Returns:
        loss: Test loss
    """
    model.eval()
    
    # Create missing mask
    mask = create_missing_mask(test_tensor, missing_fraction, masking_strategy)
    
    # Apply mask to input data
    input_with_mask = test_tensor.clone()
    
    # Use column means for missing values
    col_means = torch.mean(test_tensor, dim=0, keepdim=True)
    input_with_mask[mask == 1] = col_means.expand_as(test_tensor)[mask == 1]
    
    with torch.no_grad():
        predictions = model(input_with_mask, column_indices)
        loss = criterion(predictions, test_tensor, mask)
        
    return loss.item()

In [32]:
def train_model_with_batches(model, train_tensor, criterion, optimizer, scheduler, 
                             column_indices, num_epochs=50, batch_size=32, 
                             missing_fraction=0.2, clip_value=1.0, 
                             masking_strategy="random", 
                             validation_tensor=None, patience=5):
    """
    Train the model with batches and advanced training features.
    
    Args:
        model: Model to train
        train_tensor: Training data
        criterion: Loss function
        optimizer: Optimizer
        scheduler: Learning rate scheduler
        column_indices: Feature indices
        num_epochs: Number of training epochs
        batch_size: Batch size
        missing_fraction: Fraction of values to mask
        clip_value: Gradient clipping value
        masking_strategy: Strategy for creating missing values
        validation_tensor: Optional validation data
        patience: Early stopping patience
        
    Returns:
        model: Trained model
        train_losses: List of training losses
        val_losses: List of validation losses
    """
    # Create dataset and dataloader
    train_dataset = TensorDataset(train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    device = train_tensor.device
    
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    patience_counter = 0
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        
        # Use tqdm for progress bar
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        
        for batch in progress_bar:
            optimizer.zero_grad()
            
            # Get batch data
            batch_data = batch[0]
            
            # Create missing mask for this batch
            mask = create_missing_mask(batch_data, missing_fraction, masking_strategy)
            
            # Apply mask to input data
            input_with_mask = batch_data.clone()
            
            # More sophisticated missing value handling
            # Instead of replacing with zeros, use column means
            col_means = torch.mean(batch_data, dim=0, keepdim=True)
            input_with_mask[mask == 1] = col_means.expand_as(batch_data)[mask == 1]
            
            # Forward pass
            predictions = model(input_with_mask, column_indices)
            
            # Compute loss
            loss = criterion(predictions, batch_data, mask)
            
            # Backward pass
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            
            # Update weights
            optimizer.step()
            
            # Update progress bar
            batch_loss = loss.item()
            epoch_loss += batch_loss
            progress_bar.set_postfix({"batch_loss": f"{batch_loss:.4f}"})
        
        # Calculate average epoch loss
        avg_epoch_loss = epoch_loss / len(train_loader)
        train_losses.append(avg_epoch_loss)
        
        # Validate if validation data is provided
        if validation_tensor is not None:
            val_loss = evaluate_model(model, validation_tensor, criterion, column_indices, missing_fraction)
            val_losses.append(val_loss)
            
            # Update learning rate based on validation loss
            scheduler.step(val_loss)
            
            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                # Save best model
                torch.save(model.state_dict(), "best_model.pth")
                patience_counter = 0
            else:
                patience_counter += 1
                
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                # Load best model
                model.load_state_dict(torch.load("best_model.pth"))
                break
                
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_epoch_loss:.4f}, Val Loss: {val_loss:.4f}")
        else:
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_epoch_loss:.4f}")
            
    return model, train_losses, val_losses


In [33]:
def compute_nrmse(predictions, ground_truth, mask):
    """
    Compute Normalized Root Mean Square Error.
    
    Args:
        predictions: Model predictions
        ground_truth: True values
        mask: Binary mask where 1 indicates missing/masked values
        
    Returns:
        nrmse: Normalized RMSE
    """
    # Extract masked predictions and ground truth
    masked_predictions = predictions[mask == 1]
    masked_ground_truth = ground_truth[mask == 1]
    
    # Handle empty mask
    if masked_predictions.numel() == 0:
        return 0.0
    
    # Compute MSE and RMSE
    mse = torch.mean((masked_predictions - masked_ground_truth) ** 2)
    rmse = torch.sqrt(mse)
    
    # Compute data range for normalization
    data_range = ground_truth.max() - ground_truth.min()
    
    # Normalize RMSE
    nrmse = rmse / data_range
    
    return nrmse.item()


In [34]:
def run_enhanced_imputation_pipeline(file_path, scaler_type="robust", num_epochs=50, 
                                      batch_size=32, d_model=128, num_heads=8, 
                                      num_layers=4, dim_feedforward=512, 
                                      learning_rate=1e-3, missing_fraction=0.2,
                                      masking_strategy="random"):
    """
    Run the complete enhanced imputation pipeline.
    
    Args:
        file_path: Path to the CSV file
        scaler_type: Type of scaler to use
        num_epochs: Number of training epochs
        batch_size: Batch size for training
        d_model: Embedding dimension
        num_heads: Number of attention heads
        num_layers: Number of transformer layers
        dim_feedforward: Dimension of feedforward network
        learning_rate: Initial learning rate
        missing_fraction: Fraction of values to mask
        masking_strategy: Strategy for masking values
        
    Returns:
        model: Trained model
        train_losses: List of training losses
        val_losses: List of validation losses
        test_nrmse: NRMSE on test data
        test_tensor: Test data tensor
        column_indices: Feature indices
    """
    # Load and preprocess data
    train_tensor, test_tensor, scaler, device = load_and_preprocess_data(
        file_path, scaler_type=scaler_type
    )
    
    # Split train data into train and validation
    val_size = int(0.1 * train_tensor.shape[0])
    indices = torch.randperm(train_tensor.shape[0])
    train_indices = indices[val_size:]
    val_indices = indices[:val_size]
    
    train_data = train_tensor[train_indices]
    val_data = train_tensor[val_indices]
    
    # Create column indices
    num_features = train_data.shape[1]
    column_indices = torch.arange(num_features).to(device)
    
    # Initialize model
    model = EnhancedTransformerModel(
        num_features=num_features,
        d_model=d_model,
        num_heads=num_heads,
        num_layers=num_layers,
        dim_feedforward=dim_feedforward
    ).to(device)
    
    # Initialize loss function
    criterion = EnhancedImputableLoss(alpha=1.0, beta=0.1)
    
    # Initialize optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    
    # Initialize learning rate scheduler
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
    
    # Train model
    model, train_losses, val_losses = train_model_with_batches(
        model=model,
        train_tensor=train_data,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        column_indices=column_indices,
        num_epochs=num_epochs,
        batch_size=batch_size,
        missing_fraction=missing_fraction,
        masking_strategy=masking_strategy,
        validation_tensor=val_data,
        patience=7
    )
    
    # Evaluate on test data
    model.eval()
    mask = create_missing_mask(test_tensor, missing_fraction, masking_strategy)
    input_with_mask = test_tensor.clone()
    
    # Use column means for missing values
    col_means = torch.mean(test_tensor, dim=0, keepdim=True)
    input_with_mask[mask == 1] = col_means.expand_as(test_tensor)[mask == 1]
    
    with torch.no_grad():
        predictions = model(input_with_mask, column_indices)
        
    test_nrmse = compute_nrmse(predictions, test_tensor, mask)
    print(f"Test NRMSE: {test_nrmse:.4f}")
    
    # Plot training and validation losses
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Training Loss')
    if val_losses:
        plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.savefig('training_loss.png')
    plt.close()
    
    # Save final model
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scaler': scaler,
        'num_features': num_features,
        'config': {
            'd_model': d_model,
            'num_heads': num_heads,
            'num_layers': num_layers,
            'dim_feedforward': dim_feedforward,
        }
    }, "enhanced_tabular_transformer.pth")
    
    # Return test_tensor and column_indices as well
    return model, train_losses, val_losses, test_nrmse, test_tensor, column_indices

In [35]:
def evaluate_across_mechanisms(model, test_tensor, column_indices, missing_fractions):
    """
    Evaluate model across different missing data mechanisms and fractions.
    
    Args:
        model: Trained model
        test_tensor: Test data
        column_indices: Feature indices
        missing_fractions: List of missing data fractions to evaluate
        
    Returns:
        results: Dictionary of results
    """
    mechanisms = ["random", "structured", "column"]  # Corresponding to MCAR, MNAR, MAR
    results = {mechanism: {} for mechanism in mechanisms}
    
    model.eval()
    
    for mechanism in mechanisms:
        for fraction in missing_fractions:
            # Create missing mask
            mask = create_missing_mask(test_tensor, fraction, mechanism)
            
            # Apply mask to input data
            input_with_mask = test_tensor.clone()
            
            # Use column means for missing values
            col_means = torch.mean(test_tensor, dim=0, keepdim=True)
            input_with_mask[mask == 1] = col_means.expand_as(test_tensor)[mask == 1]
            
            with torch.no_grad():
                predictions = model(input_with_mask, column_indices)
                
            nrmse = compute_nrmse(predictions, test_tensor, mask)
            results[mechanism][fraction] = nrmse
            
            print(f"{mechanism.upper()} NRMSE at {fraction * 100:.0f}% Missing: {nrmse:.4f}")
    
    return results

In [36]:
def run_evaluation_example(file_path):
    # Run the enhanced pipeline and capture all return values including test_tensor and column_indices
    model, train_losses, val_losses, test_nrmse, test_tensor, column_indices = run_enhanced_imputation_pipeline(
        file_path=file_path,
        scaler_type="robust",
        num_epochs=50,
        batch_size=32,
        d_model=128,
        num_heads=8,
        num_layers=4,
        dim_feedforward=512,
        learning_rate=1e-3,
        missing_fraction=0.2,
        masking_strategy="random"
    )
    
    # Now we have test_tensor and column_indices available for the evaluation
    missing_fractions = [0.1, 0.2, 0.3, 0.4, 0.5]
    results = evaluate_across_mechanisms(model, test_tensor, column_indices, missing_fractions)
    
    # Print results
    for mechanism, nrmse_values in results.items():
        print(f"\n{mechanism.upper()} Results:")
        for frac, nrmse in nrmse_values.items():
            print(f"  Missing Fraction {frac * 100:.0f}%: NRMSE = {nrmse:.4f}")
    
    return model, results

In [37]:
if __name__ == "__main__":
    # Run the evaluation with the correct file path
    run_evaluation_example(file_path='../data/physionet_wo_missing.csv')

c:\Users\wh1sper\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Epoch 1/50: 100%|██████████| 36/36 [00:02<00:00, 17.21it/s, batch_loss=0.7498]  


Epoch 1/50, Train Loss: 14.4135, Val Loss: 1.0541


Epoch 2/50: 100%|██████████| 36/36 [00:02<00:00, 17.47it/s, batch_loss=5.1065] 


Epoch 2/50, Train Loss: 2.5376, Val Loss: 1.2830


Epoch 3/50: 100%|██████████| 36/36 [00:02<00:00, 17.95it/s, batch_loss=19.6074]


...

Epoch 11/50, Train Loss: 16.3153, Val Loss: 1.1013


Epoch 12/50: 100%|██████████| 36/36 [00:02<00:00, 17.67it/s, batch_loss=10.7853]


Epoch 12/50, Train Loss: 2.3375, Val Loss: 0.8992


Epoch 13/50: 100%|██████████| 36/36 [00:02<00:00, 17.83it/s, batch_loss=0.3334] 


Early stopping at epoch 13
Test NRMSE: 0.0036
RANDOM NRMSE at 10% Missing: 0.0023
RANDOM NRMSE at 20% Missing: 0.0116
RANDOM NRMSE at 30% Missing: 0.0043
RANDOM NRMSE at 40% Missing: 0.0085
RANDOM NRMSE at 50% Missing: 0.0085
STRUCTURED NRMSE at 10% Missing: 0.0089
STRUCTURED NRMSE at 20% Missing: 0.0199
STRUCTURED NRMSE at 30% Missing: 0.0165
STRUCTURED NRMSE at 40% Missing: 0.0169
STRUCTURED NRMSE at 50% Missing: 0.0153
COLUMN NRMSE at 10% Missing: 0.0085
COLUMN NRMSE at 20% Missing: 0.0040
COLUMN NRMSE at 30% Missing: 0.0095
COLUMN NRMSE at 40% Missing: 0.0082
COLUMN NRMSE at 50% Missing: 0.0150

RANDOM Results:
  Missing Fraction 10%: NRMSE = 0.0023
  Missing Fraction 20%: NRMSE = 0.0116
  Missing Fraction 30%: NRMSE = 0.0043
  Missing Fraction 40%: NRMSE = 0.0085
  Missing Fraction 50%: NRMSE = 0.0085

STRUCTURED Results:
  Missing Fraction 10%: NRMSE = 0.0089
  Missing Fraction 20%: NRMSE = 0.0199
  Missing Fraction 30%: NRMSE = 0.0165
  Missing Fraction 40%: NRMSE = 0.0169
  Mis